In [1]:
# %pip install huggingface_hub transformers llama_index nbdev

In [2]:
from dataclasses import dataclass, asdict

from huggingface_hub import hf_hub_download, snapshot_download
from transformers import AutoTokenizer

from nbdev.showdoc import patch_to

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
import os



def upsert_folder(folder_path: str, debug_prn: bool = False):

    folder_path = os.path.dirname(folder_path)


    if debug_prn:

        print(
            {
                "upsert_folder": os.path.abspath(folder_path),
                "is_exist": os.path.exists(folder_path),
            }
        )


    if not os.path.exists(folder_path):

        os.makedirs(folder_path)

In [4]:
@dataclass
class ModelPath:
    install_path: str
    cache_path: str = None
    embedding_path: str = None

    def __post_init__(self):
        self.cache_path = self.cache_path or f"{self.install_path}/cache/"
        self.embedding_path = self.embedding_path or f"{self.install_path}/embedding/"

In [5]:
asdict(ModelPath(install_path="../../models"))

{'install_path': '../../models',
 'cache_path': '../../models/cache/',
 'embedding_path': '../../models/embedding/'}

In [6]:
@dataclass
class ModelConfig:
    install_path: str

    prompt_style: str = "llama2"
    llm_hf_repo_id: str = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
    llm_tokenizer_name: str = "mistralai/Mistral-7B-Instruct-v0.1"
    llm_hf_model_file: str = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"
    embedding_hf_model_name: str = "BAAI/bge-small-en-v1.5"

    model_paths: ModelPath = None

    def __post_init__(self):
        self.model_paths = ModelPath(install_path=self.install_path)

In [7]:
@patch_to(ModelConfig)
def setup_model_folders(self, debug_prn: bool = False):
    [
        upsert_folder(folder_path, debug_prn=debug_prn)
        for folder_path in asdict(self.model_paths).values()
    ]

    return True

In [8]:
config = ModelConfig(install_path='./models/', 
    llm_hf_repo_id  = "TheBloke/gorilla-openfunctions-v1-GGUF",
    llm_hf_model_file  = "gorilla-openfunctions-v1.Q8_0.gguf",
)

config.setup_model_folders( debug_prn = True)


{'upsert_folder': 'c:\\Users\\jaewi\\GitHub\\llama\\models', 'is_exist': True}
{'upsert_folder': 'c:\\Users\\jaewi\\GitHub\\llama\\models\\cache', 'is_exist': True}
{'upsert_folder': 'c:\\Users\\jaewi\\GitHub\\llama\\models\\embedding', 'is_exist': True}


True

In [9]:
# | exporti
@patch_to(ModelConfig)
def download_embeddings_model(self):
    print(f"Downloading embedding {self.embedding_hf_model_name}")

    snapshot_download(
        repo_id=self.embedding_hf_model_name,
        cache_dir=self.model_paths.cache_path,
        local_dir=self.model_paths.embedding_path,
    )
    print(f"Embedding model downloaded")

In [10]:
config.download_embeddings_model()

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Embedding model downloaded


c:\Users\jaewi\GitHub\llama\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jaewi\GitHub\llama\models. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
@patch_to(ModelConfig)
def download_llm_model(self, resume_download: bool = True):
    """Download LLM and create a symlink to the model file"""
    print(f"Downloading LLM {self.llm_hf_model_file}")

    hf_hub_download(
        repo_id=self.llm_hf_repo_id,
        filename=self.llm_hf_model_file,
        cache_dir=self.model_paths.cache_path,
        local_dir=self.model_paths.install_path,
        resume_download=resume_download,
    )
    print("LLM model downloaded!")

In [12]:
config.download_llm_model()

gorilla-openfunctions-v1.Q8_0.gguf:   0%|          | 0.00/7.16G [00:00<?, ?B/s]

LLM model downloaded!


In [13]:
@patch_to(ModelConfig)
def download_tokenizer(self):
    print(f"Downloading tokenizer {self.llm_tokenizer_name}")

    AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=self.llm_tokenizer_name,
        cache_dir=self.model_paths.cache_path,
    )
    print("Tokenizer downloaded!")

In [14]:
config.download_tokenizer()

Tokenizer downloaded!


In [15]:
@patch_to(ModelConfig)
def setup(self):
    self.download_embeddings_model()
    self.download_llm_model()
    self.download_tokenizer()

In [16]:
config.setup()

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Embedding model downloaded
LLM model downloaded!
Tokenizer downloaded!
